In [35]:
from fastapi import FastAPI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
import os

from dotenv import load_dotenv

# Load environment variables
load_dotenv()


groq_api_key=os.getenv("GROQ_API_KEY")
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)


genric_template="tell car of this brand"

prompt=ChatPromptTemplate.from_messages(
    [("system",genric_template),("user","{text}")]
)

parser=StrOutputParser()

In [36]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi,my name is yash")])

AIMessage(content="Hi Yash, it's nice to meet you! What can I do for you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 16, 'total_tokens': 41, 'completion_time': 0.045454545, 'prompt_time': 7.665e-05, 'queue_time': 0.004353129, 'total_time': 0.045531195}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-816e0ca1-fb7b-49fb-9fe5-11c1b3261b3a-0', usage_metadata={'input_tokens': 16, 'output_tokens': 25, 'total_tokens': 41})

In [37]:
from langchain_core.messages import AIMessage
model.invoke([
    HumanMessage(content="Hi,my name is yash"),
    AIMessage(content="Hello,yash! How are you today?"),
    HumanMessage(content="hey whats my name ?")]
)

AIMessage(content="Your name is Yash, that's what you told me! 😊  \n\nIs there anything else I can help you with?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 41, 'total_tokens': 70, 'completion_time': 0.052727273, 'prompt_time': 0.000544348, 'queue_time': 0.003679872, 'total_time': 0.053271621}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-65fcd4f2-bc9e-40f9-afc5-2f99278ae455-0', usage_metadata={'input_tokens': 41, 'output_tokens': 29, 'total_tokens': 70})

In [38]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}
def get_seess_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory
    return store[session_id]    
        
        
with_message_history=RunnableWithMessageHistory(runnable=model,
    get_session_history=get_seess_history)        

In [39]:
config = {
    "configurable": {
        "session_id":"chat1"  # Replace with your session ID
    }
}


In [41]:
res=with_message_history.invoke(
    [HumanMessage(content="hi iam Yash")],
    config=config,
)

AttributeError: messages

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage

# A dictionary to store session histories
store = {}

# Function to get or create a session history
def get_seess_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        # Initialize an instance of ChatMessageHistory and store it
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Example: Dummy model object (replace with your actual model)
from langchain_core.runnables import RunnablePassthrough
model = RunnablePassthrough()  # Replace with your actual model instance

# Create the RunnableWithMessageHistory
with_message_history = RunnableWithMessageHistory(
    runnable=model,
    get_session_history=get_seess_history
)

# Example usage
session_id = "session_1"
history = get_seess_history(session_id)
history.add_message(HumanMessage(content="Hi, I am Yash."))

# Config to include session ID
config = {"configurable": {"session_id": session_id}}

# Invoke with message history
response = with_message_history.invoke(
    [HumanMessage(content="What can you do?")],
    config=config,
)

response.content


AttributeError: 'list' object has no attribute 'content'

In [33]:
session_id = "chat2"
config = {"configurable": {"session_id": session_id}}

# Invoke with message history
res = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,  # Use the corrected config here
)

print(res.content)


Error in RootListenersTracer.on_chain_end callback: ValueError('Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got messages=[SystemMessage(content=\'You are a helpful assistant.\', additional_kwargs={}, response_metadata={}), HumanMessage(content="What\'s my name?", additional_kwargs={}, response_metadata={}), HumanMessage(content="What\'s my name?", additional_kwargs={}, response_metadata={}), HumanMessage(content="What\'s my name?", additional_kwargs={}, response_metadata={}), HumanMessage(content="What\'s my name?", additional_kwargs={}, response_metadata={}), HumanMessage(content="What\'s my name?", additional_kwargs={}, response_metadata={}), HumanMessage(content="What\'s my name?", additional_kwargs={}, response_metadata={}), HumanMessage(content="What\'s my name?", additional_kwargs={}, response_metadata={})].')


AttributeError: 'ChatPromptValue' object has no attribute 'content'

In [22]:
res.content

AttributeError: 'list' object has no attribute 'content'

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, MessagesPlaceholder

# Define the chat prompt template
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template("You are a helpful assistant."),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

# Combine the prompt with the model
chain = prompt | model


In [26]:
chain.invoke({"messages":[HumanMessage(content="hi my name is yash")]})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi my name is yash', additional_kwargs={}, response_metadata={})])

In [27]:
with_message_history=RunnableWithMessageHistory(chain,get_seess_history)

In [29]:
config={"configurable":{"session_id":"chat3"}}
res=with_message_history.invoke(
    [HumanMessage(content="hi my name is yash")],
    config=config
)

Error in RootListenersTracer.on_chain_end callback: ValueError("Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got messages=[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi my name is yash', additional_kwargs={}, response_metadata={})].")


In [31]:
print(res)

messages=[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi my name is yash', additional_kwargs={}, response_metadata={})]


In [32]:
res

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi my name is yash', additional_kwargs={}, response_metadata={})])

In [50]:
from langchain_core.messages import SystemMessage, trim_messages

# Trimmer setup
trimmer = trim_messages(
    max_tokens=70,
    strategy="last",
    token_counter=model,  # Ensure the `model` can count tokens
    include_system=True,
    allow_partial=False,
    start_on="human",
)

# Define messages list (you can populate it as needed)
messages = [
    SystemMessage(content="You are a helpful assistant."),
    # Add HumanMessage, AIMessage, etc., to simulate a conversation
    HumanMessage(content="hi iam yash"),
    AIMessage(content="hi")
]
trimmer.invoke(messages)


[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi iam yash', additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi', additional_kwargs={}, response_metadata={})]

In [52]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)|
    prompt|model
)
res=chain.invoke(
    "messages":messages+[HumanMessage(content="what i like")],
    "language":"English"
)
res.content

SyntaxError: invalid syntax (4198636768.py, line 9)